In [1]:
import keras
from keras.datasets import mnist
from keras.utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
import numpy as np
from sklearn.model_selection import train_test_split

import librosa 
from librosa import feature
import soundfile
import os, glob, pickle
import re #split strings

from sklearn import preprocessing

In [2]:
fn_list_i = [
 feature.chroma_stft,
 feature.spectral_centroid,
 feature.spectral_bandwidth,
 feature.spectral_rolloff
]
 
fn_list_ii = [
 feature.zero_crossing_rate
]
#feature.rmse,


In [3]:
emotions={
    0:'neutral',
    1:'calm',
    2:'happy',
    3:'sad',
    4:'angry',
    5:'fearful',
    6:'disgust',
    7:'surprised'
}

In [4]:
# Not stressed: happy(2), calm(1).
# Stressed:     sad(3), fearful(5)
stress_emotions = {
    1:0,
    2:0,
    3:1,
    5:1
}
selected_emotions = {1,2,3,5}


num_classes = 2 #8

from sklearn import preprocessing

In [5]:
#preprocesamiento de corpus
def preprocessing_data(filename):
    # 1) audio: mono 
    # 2) frecuencia de muestreo(SR): 16000Hz
    data, sr = librosa.load(filename, sr=16000, mono=True)
    #3) normalizacion
    X_scale = preprocessing.normalize([data])[0]
    #plot_data(normalizado, sr)
    #TODO: otros: farming, windowing, Voice Activity Detector (VAD), noise reduction,
    return [X_scale, sr]

In [6]:
def TEO(signal):
    out = []
    for i in range(0, len(signal)):
        if i == 0 or i == len(signal) - 1:
            out.append(signal[i])
        else:
            out.append(pow(signal[i], 2) - (signal[i + 1] * signal[i - 1]))
    return out

In [7]:
def get_feature_vector(signal,sample_rate): 
    feat_vect_i = [ np.mean(funct(signal,sample_rate)) for funct in fn_list_i]
    feat_vect_ii = [ np.mean(funct(signal)) for funct in fn_list_ii] 
    
    feature_vector = np.array([])
    mfccs = np.mean(librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=40).T, axis=0)
    mfccs_vect_iii = np.hstack((feature_vector, mfccs)) 

    feature_vector_var = np.array([])
    mfccs_var = np.var(librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=40).T, axis=0)
    mfccs_vect_var = np.hstack((feature_vector_var, mfccs_var)) 

    feature_vector_std = np.array([])
    mfccs_std = np.std(librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=40).T, axis=0)
    mfccs_vect_std = np.hstack((feature_vector_std, mfccs_std)) 

    feature_vector = feat_vect_i + feat_vect_ii + mfccs_vect_iii.tolist() + mfccs_vect_var.tolist() + mfccs_vect_std.tolist()
    return feature_vector 

In [8]:
def load_data(test_size=0.2):
    x,y=[],[]
    #for file in glob.glob("/Users/y/Documents/datasets/RAVDESS/ravdess_test/Actor_*/*.wav"): #* LAP
    for file in glob.glob("/media/y/730D-8298/DATASETS/dataFlair/RAVDESS/Actor_*/*.wav"): #* PC
        file_name = os.path.basename(file) #            03-01-01-01-01-01-01.wav
        print('Progress:  %s' % (file_name), end='\r')
        file_name_no_type = file_name.split(".")[0] #   03-01-01-01-01-01-01
        list_emotions = file_name_no_type.split("-") #  ['03', '01', '01', '01', '01', '01', '01']
        emotion = int(list_emotions[2])-1 #iniciar contador de emociones desde 0 a 7 (8emociones)
        
        if emotion in selected_emotions:
            emotion = stress_emotions[emotion]
           
            signal, sample_rate = preprocessing_data(file)
            feature = get_feature_vector(signal, sample_rate)#extract_feature(file, mfcc=True) #TODO: add more features
            x.append(feature)
            y.append(emotion)

    return np.array(x), np.array(y)

In [9]:
min_max_scaler = preprocessing.MinMaxScaler()
X_raw,Y_raw = load_data()

In [10]:
print("X_data:", X_raw.shape, "Y_data:",  Y_raw.shape)

Y = to_categorical(Y_raw, num_classes)
X = min_max_scaler.fit_transform(X_raw)
print("X_data:", X.shape, "Y_data:",  Y.shape)

X_data: (768, 125) Y_data: (768,)
X_data: (768, 125) Y_data: (768, 2)


In [11]:
X_train, X_val_test, Y_train, Y_val_test = train_test_split(X,Y, test_size=0.3, random_state=30) 
X_val, X_test, Y_val, Y_test = train_test_split(X_val_test,Y_val_test, test_size=0.5, random_state=20)
print("Train:", X_train.shape, Y_train.shape, "Val:", X_val.shape, Y_val.shape, "Test:", X_test.shape, Y_test.shape)


Train: (537, 125) (537, 2) Val: (115, 125) (115, 2) Test: (116, 125) (116, 2)


In [14]:
# Modelo
model = Sequential()
model.add(Dense(350, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


In [15]:
# Entrenamiento y validacion
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=64, epochs=100, verbose=1, validation_data=(X_val, Y_val)) #verbose=1
 

Epoch 1/100
9/9 [==============================] - 0s 26ms/step - loss: 0.6948 - accuracy: 0.5233 - val_loss: 0.6743 - val_accuracy: 0.6087
Epoch 2/100
9/9 [==============================] - 0s 10ms/step - loss: 0.6710 - accuracy: 0.6015 - val_loss: 0.6837 - val_accuracy: 0.5391
Epoch 3/100
9/9 [==============================] - 0s 10ms/step - loss: 0.6736 - accuracy: 0.5456 - val_loss: 0.6579 - val_accuracy: 0.6087
Epoch 4/100
9/9 [==============================] - 0s 10ms/step - loss: 0.6451 - accuracy: 0.6387 - val_loss: 0.6453 - val_accuracy: 0.5913
Epoch 5/100
9/9 [==============================] - 0s 10ms/step - loss: 0.6321 - accuracy: 0.6536 - val_loss: 0.6384 - val_accuracy: 0.6261
Epoch 6/100
9/9 [==============================] - 0s 11ms/step - loss: 0.6199 - accuracy: 0.6816 - val_loss: 0.6241 - val_accuracy: 0.6261
Epoch 7/100
9/9 [==============================] - 0s 9ms/step - loss: 0.6063 - accuracy: 0.6983 - val_loss: 0.6077 - val_accuracy: 0.6696
Epoch 8/100
9/9 [====

In [16]:
# Prueba
print("Exactitud")
print(model.evaluate(X_test,Y_test)[1],"%") #[0] perdida, [1] presicion

Exactitud
4/4 [==============================] - 0s 6ms/step - loss: 0.4765 - accuracy: 0.8534
0.8534482717514038 %
